In [1]:
# Torch e CUDA
import torch
import gc
from torch.utils.data import Subset

# Transformers e Training
from transformers import (
    TextStreamer,
    TrainingArguments,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model
# Dataset e valutazione
from datasets import load_dataset, Dataset
from evaluate import load
import bitsandbytes as bnb
# Metriche di valutazione
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    confusion_matrix
)

# Data manipulation
import pandas as pd
import numpy as np

# Sistema e utility
import os
from dotenv import load_dotenv
from pathlib import Path
from datetime import datetime

# Visualizzazione
import seaborn as sns
import matplotlib.pyplot as plt


/home/raffaeleterracino/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


prompt

In [2]:
prompt_template = (
    "### Instruction:\n"
    "You are an expert software developer and bug triaging specialist. Your task is to predict whether a bug "
    "will be resolved in LESS than 50 DAYS or MORE than 50 DAYS based on the provided bug details.\n\n"
    
    "- Output '0' if the bug will be resolved in LESS than 50 DAYS.\n"
    "- Output '1' if the bug will be resolved in MORE than 50 DAYS.\n\n"
    
    "Your response MUST be strictly either '0' or '1'. Do NOT include any additional text, explanations, formatting, symbols, or extra characters in your response.\n\n"

    "### Input:\n"
    "Source: {source}\n"
    "Product: {product}"
    "Short Description: {short_desc}\n"
    "Priority: {priority}\n"
    "Severity: {bug_severity}\n"
    #"Estimated resolution time: {days_resolution}\n\n" - questo potrebbe influenzare troppo il modello per la predizione

    "### Example Responses:\n"
    "Input: Source: KDE | Product: Payment System | Short Description: Critical security vulnerability found in authentication system | Priority: P1 | Severity: Critical\n"
    "Output: 0\n\n"
    "Input: Source: OpenOffice | Product: UI Module | Short Description: UI glitch affecting low-impact visual elements in settings panel | Priority: P3 | Severity: Minor\n"
    "Output: 1\n\n"

    "### Output: {label}\n"
)
num_val = "2000" #1000, 2000, 5000, 9000

caricamento del modello

In [3]:
max_seq_length = 2048
dtype = torch.float16 #altrimenti None
load_in_4bit = True
seed = 3407
load_dotenv()
hf_token = os.getenv("HF_TOKEN")
model_name="meta-llama/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name, token = hf_token)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)


Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.27s/it]


In [4]:
# Peft -  Parameter Efficient Fine Tuning
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    # Layer fondamentali per catturare relazioni tra token
    # q_proj : "Query projection", v_proj: "Value projection", k_proj : "Key projection", o_proh: "output projection"
    #target_modules=['q_proj', 'v_proj', 'k_proj','o_proj','gate_proj','up_proj','down_proj','lm_head','embedded_layers']
    target_modules = ['q_proj', 'v_proj', 'gate_proj', 'up_proj', 'down_proj'] #forse lm_head non serve perchè generiamo solo un singolo token
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

trainable params: 35,127,296 || all params: 8,065,388,544 || trainable%: 0.4355


formattazione del prompt con i dati del dataset

In [5]:
EOS_TOKEN = tokenizer.eos_token  # Assicuriamoci di aggiungere il token EOS alla fine

def formatting_prompts(examples, include_label=True):
    texts = []
    for source, product, short_desc, priority, bug_severity, label in zip(
        examples["source"],examples["product"], examples["short_desc"], examples["priority"], examples["bug_severity"], examples["label"]
    ):
        if include_label:
            text = prompt_template.format(
                source=source,
                product=product,
                short_desc=short_desc,
                priority=priority,
                bug_severity=bug_severity,
                label=label,  # La label viene passata solo se include_label=True
            ) + EOS_TOKEN
        else:
            text = prompt_template.format(
                source=source,
                product=product, 
                short_desc=short_desc,
                priority=priority,
                bug_severity=bug_severity,
                label="",  #  Non passiamo la label
            ) + EOS_TOKEN
        
        texts.append(text)
    
    return {"text": texts}


# Caricamento dataset
dataset = load_dataset(
    "csv",
    data_files={
        "train": f"../dataset_completo/balanced_datasets/balanced_train_{num_val}.csv", 
        "test": f"../dataset_completo/balanced_datasets/balanced_test.csv", 
        "val": f"../dataset_completo/balanced_datasets/balanced_validation.csv" 
    },
)

# Formattiamo il dataset con il nuovo prompt
# Applichiamo la funzione al dataset
dataset["train"] = dataset["train"].map(lambda x: formatting_prompts(x, include_label=True), batched=True)
dataset["val"] = dataset["val"].map(lambda x: formatting_prompts(x, include_label=False), batched=True)  # 🚨 Label nascosta
#dataset["test"] = dataset["test"].map(lambda x: formatting_prompts(x, include_label=False), batched=True)  # 🚨 Label nascosta


dataset['train'][0]

Generating train split: 1998 examples [00:00, 57397.98 examples/s]
Generating test split: 2250 examples [00:00, 59911.02 examples/s]
Generating val split: 2250 examples [00:00, 64019.54 examples/s]
Map: 100%|██████████| 2250/2250 [00:00<00:00, 115494.66 examples/s]


{'short_desc': 'i965 BisectedOglc apierrornegativeglEvalMesh fails',
 'product': 'Mesa',
 'priority': 'high',
 'bug_severity': 'major',
 'days_resolution': 28,
 'comments': 'System Environment Arch x8664 Platform Ivyybridge Libdrmmasterlibdrm24413g303ca37e722e68900cb7eb43ddbef8069b0c711b Mesamaster85c2e99039394292474b1a84e3dcb2fee30a0836 Xservermasterxorgserver1139990136g70b127c9f1c53bdb42f078265e67f76b464deae2 Xf86videointelmaster2201914g10f549332e315cfe2cc86aadab94a95ae6757c34 Cairomastered2fa6b16b03fccc3e21598cdb9157cbcebd1d37 Libvastaging21649988d6b532cc96f633db017d1e4369f640e9 Libvainteldriverstagingd206b47a6ac86c089149ecd71b01eea6ebda5796 Kerneldrmintelnightly 418ccc855c65e0e90f81012bbc34de20b9f45cbd Bug detailed description It fails on ironlake sandybridge and ivybridge with mesa master branch It works well on mesa 90 branch Bisect shows a9754793dab4b24c09cae21c29f902ce0e53319a is the first bad commit commit a9754793dab4b24c09cae21c29f902ce0e53319a Author Eric Anholt AuthorDate 

Fine-tuning del modello

In [ ]:
from evaluate import load
from trl import SFTTrainer, SFTConfig
# Carichiamo la metrica di accuracy
#accuracy_metric = load("accuracy")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # ✅ Fix padding issue
    tokenizer.padding_side = "right"
model.train() 
directory = f"{model_name}".split("/")[-1].strip()
# 🔹 Configurazione per l'addestramento (usando SFTConfig)
sft_config = SFTConfig(
    output_dir=f"{directory}_{num_val}_ft",
    max_seq_length=2048,
    dataset_text_field="text",  # Cambia se necessario
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=5,  #  Più epoche per adattare bene LoRA
    gradient_accumulation_steps=4,  #  Ridotto per aggiornamenti più frequenti
    evaluation_strategy="steps",  #  Valutazione più frequente
    eval_steps=100, 
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=3,  #  Evita troppi checkpoint
    learning_rate=5e-5,  #  Aumentato per migliorare adattamento
    lr_scheduler_type="cosine",  #  Cosine decay per convergenza più fluida
    warmup_ratio=0.05,  # Warmup ridotto per velocizzare training
    fp16=True,  #  Mantieni mixed precision
    logging_steps=50,  #  Meno logging per ridurre overhead
    metric_for_best_model="eval_loss",  # 👈 Assicura che il modello salvi in base alla Validation Loss
    greater_is_better=False  # 👈 Perché una loss minore è meglio
)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    peft_config=peft_config,
    max_seq_length=2048,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=sft_config,
    packing= False,
)

# Avviamo il training!
trainer_stats = trainer.train()
eval_results = trainer.evaluate()
print(trainer_stats)
print(eval_results)

/home/raffaeleterracino/.local/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/raffaeleterracino/.local/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/raffaeleterracino/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/home/raffaeleterracino/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a

Step,Training Loss,Validation Loss


In [ ]:
import pandas as pd
from pathlib import Path

# Definiamo le metriche da salvare
training_results = {
    "Dataset Size": num_val,  # Numero di dati usati per il fine-tuning
    "Training Loss": trainer_stats.training_loss,  # Training Loss
    "Train Time (s)": trainer_stats.metrics["train_runtime"],  # Tempo di addestramento
    "Steps": trainer_stats.global_step,  # Numero di passi (steps)
    "Samples/sec": trainer_stats.metrics["train_samples_per_second"],  # Campioni al secondo
    "Steps/sec": trainer_stats.metrics["train_steps_per_second"],  # Passi al secondo
    "Validation Loss": eval_results.get("eval_loss", None),  # Valutazione della loss
}

# Definiamo il file di destinazione per i risultati
results_file = f"{model_name}_fine_tuned_on_{num_val}/training_comparison.csv"

# Converti il dizionario in un DataFrame
training_results_df = pd.DataFrame([training_results])  # Passiamo una lista contenente il dizionario

# Assicura che la cartella esista prima di salvare
Path(results_file).parent.mkdir(parents=True, exist_ok=True)

# Salva il DataFrame nel file CSV, sovrascrivendo se già esistente
training_results_df.to_csv(results_file, index=False)

# Mostra la tabella aggiornata
print(training_results_df)

In [ ]:
directory = f"{model_name}".split("/")[-1].strip().lower()

model.save_pretrained(f"./fine_tuned_model_{directory}_{num_val}" )
tokenizer.save_pretrained(f"./fine_tuned_model_{directory}_{num_val}" )